This is the notebook for test_cleaning.py

In [1]:
#################################################################
#                                                               #
# Copyright 2014, Institute for Defense Analyses                #
# 4850 Mark Center Drive, Alexandria, VA; 703-845-2500          #
# This material may be reproduced by or for the US Government   #
# pursuant to the copyright license under the clauses at DFARS  #
# 252.227-7013 and 252.227-7014.                                #
#                                                               #
# LARC : Linear Algebra via Recursive Compression               #
# Authors:                                                      #
#   - Steve Cuccaro (IDA-CCS)                                   #
#   - John Daly (LPS)                                           #
#   - John Gilbert (UCSB, IDA adjunct)                          #
#   - Jenny Zito (IDA-CCS)                                      #
#                                                               #
# Additional contributors are listed in "LARCcontributors".   #
#                                                               #
# POC: Jennifer Zito <jszito@super.org>                         #
# Please contact the POC before disseminating this code.        #
#                                                               #
#################################################################

import os
import sys 
sys.path.append("../../src")
import pylarc
import numpy as np
from ctypes import *
# import matplotlib.pyplot as plt
# %matplotlib nbagg

# print "This code tests some basic matrix building and reading routines\n"
# initialize larc
mat_store_exp = 15
op_store_exp = 5
max_level = 10
rnd_sig_bits = -1   # default value
trunc_to_zero_bits = -1  # default value
pylarc.create_report_thread(1800)
pylarc.initialize_larc(mat_store_exp,op_store_exp,max_level,rnd_sig_bits,trunc_to_zero_bits)

# Define string for using in formating filenames
if pylarc.cvar.scalarTypeDef == 'i':
    scalarType = "Integer"
elif pylarc.cvar.scalarTypeDef == 'c':
    scalarType = "Complex"
elif pylarc.cvar.scalarTypeDef == 'r':
    scalarType = "Real"
else:
    raise Exception('scalarTypeDef %s was not handled.'%(pylarc.cvar.scalarTypeDef,))

# Calculate number of matrices created, then print part of matrix store
num_matrices_made = pylarc.num_matrices_created()
print "\n%d matrices have been created" %num_matrices_made
end = num_matrices_made - 1
filename = "../dat/out/preload.%s.store" %scalarType
pylarc.matrix_store_info_to_file(0,end,filename,"After preload with parameters: 26, 24, 10.")
print "\nThe initial matrix store can be seen in %s\n" %filename

# type control return to execute this code block



158 matrices have been created

The initial matrix store can be seen in ../dat/out/preload.Real.store



1

In [2]:
    
    
    # SAMPLE ACTIVITES WITH MATRIX INPUT AND OUTPUT, & LEARNING PYTHON
    verbose = 0
    
    # reading prewritten nonsquare matrices, and naive printout
    filename = "../dat/in/sample.1.2.%s.json" %scalarType
    sampmID = pylarc.matrix_read_json_file_matrixID(filename)
    if verbose:
        print "Read in json file %s\n" %filename
        print "Which is the matrix with matrixID %d\n" %sampmID
        print "Printing this matrix in standard format gives:\n"
        pylarc.print_matrix_naive_by_matrixID(sampmID)
        print "\n"
    
 
    # create a matrix in python for various scalar types: int, complex, real
    if pylarc.cvar.scalarTypeDef == 'i':
        a = np.matrix([[1, 3, 5, 6],
                       [8, 6, 3, 1],
                       [-9, 11, 13, 15],
                       [16, 13, 12, 10]])
    elif pylarc.cvar.scalarTypeDef == 'c':
        a = np.matrix([[1+2j, 3+4j, 5+6j, 7+8j],
                       [8+7j, 6+5j, 3+4j, 1+2j],
                       [9+10j, 11+12j, 13+14j, 15+16j],
                       [16+15j, 14+13j, 12+11j, 10+9j]])
    elif pylarc.cvar.scalarTypeDef == 'r':
        a = np.matrix([[1, 3, .5, 6],
                       [8, 6, 3, .1],
                       [-9, 11, 13, 1.5],
                       [16, 13, 12, 10]])
    else:
        raise Exception('Do not know how to build matrix for type %s.'%(pylarc.cvar.scalarTypeDef,))
    if verbose:
        a

        
    # convert python matrix into an array by reading off each row in turn (row major format)
    alist = a.reshape(-1).tolist()[0]
    arr = pylarc.buildArray(alist)
    if verbose:
        print 'arr:', pylarc.str_scalarTypeArray(arr, len(alist))
        
        
    # Use C routing row_major_list_to_store_matrixID to
    # interpret the array as a row major list of a matrix
    # enter it into the matrix store and then return 
    # the matrixID of the stored matrix.
    if verbose:
        print "Using row_major_list_to_store on data entered from python\n"
    # parameters for entering the python array into the store
    level = 2
    dim_whole = 2**level
    # creating or finding the matrix associated with the array
    serial = pylarc.row_major_list_to_store_matrixID(arr, level, level, dim_whole)
    if verbose:
        pylarc.print_matrix_naive_by_matrixID(serial)
        print "\n"
   

    # Make a parent matrix from four copies of the a matrix 
    # as specified by its matrixID
    if verbose:
        print "Creating matrix from matrix_get_matrixID_from_panel on panel input and writing json file\n"
    panel = [serial]*4   # alternatively panel=[serial,serial,serial,serial]
    serial_parent = pylarc.matrix_get_matrixID_from_panel(serial,serial,serial,serial,3,3)
    if verbose:
        pylarc.print_matrix_naive_by_matrixID(serial_parent)
        
        
    # Writing a matrix to a file in our compressed form (json file)
    filename = "../dat/out/testfile.%s.json" %scalarType
    pylarc.matrix_write_json_file_matrixID(serial_parent,filename)
    if verbose:
        print "Wrote a matrix constructed from four panel submatrices to the file %s\n" %filename
 

    #  Read in a compressed matrix (for the Toffoli gate)    
    filename = "../dat/in/toffoli.%s.json" %scalarType
    toffmID = pylarc.matrix_read_json_file_matrixID(filename)
    if verbose:
        print "Reading matrix file in compressed form (using json format)\n"
        print "   for the Toffoli gate from the file %s\n" %filename
        pylarc.print_matrix_naive_by_matrixID(toffmID)
        print "\n"

    # Reading and writing matrices in various formats
    if verbose:
        print "Matrices can be read or stored in many formats\n"
        print "  if small they can be written or read as row major matrices (rmm)\n"
        print "  if small they can be written in naive format\n"
        print "  if large they can be written or read in compressed format (json)\n"
        print "Testing reading row major matrix format and writing files in json and naive format.\n"
    filename_rmm = "../dat/in/sample.1.1.%s.rmm" %scalarType
    filename_naive = "../dat/out/sample.1.1.%s.naive" %scalarType
    filename_json = "../dat/out/sample.1.1.%s.json" %scalarType   
    samplemID = pylarc.read_row_major_matrix_from_file_matrixID(filename_rmm)
    pylarc.print_matrix_to_file_naive_by_matrixID(samplemID,filename_naive)
    pylarc.matrix_write_json_file_matrixID(samplemID,filename_json)
    if verbose:
        print "Printing out the rrm sample matrix in naive format to screen\n"
        pylarc.print_matrix_naive_by_matrixID(samplemID)
        print "Testing reading row major nonsquare matrices and writing files in json and naive format.\n"
    filename_rmm = "../dat/in/sample.1.2.%s.rmm" %scalarType
    filename_naive = "../dat/out/sample.1.2.%s.naive" %scalarType
    filename_json = "../dat/out/sample.1.2.%s.json" %scalarType
    samplemID = pylarc.read_row_major_matrix_from_file_matrixID(filename_rmm)
    pylarc.print_matrix_to_file_naive_by_matrixID(samplemID,filename_naive)
    pylarc.matrix_write_json_file_matrixID(samplemID,filename_json)
    if verbose:
        print "Printing out the nonsquare rrm sample matrix in naive format to screen\n"
        pylarc.print_matrix_naive_by_matrixID(samplemID)
        print "Test printing nonzeroes to file.\n"
    filename_rmm = "../dat/in/sample.1.3.%s.rmm" %scalarType
    filename_nonzeros = "../dat/out/sample.1.3.%s.nonzeros" %scalarType
    filename_json = "../dat/out/sample.1.3.%s.json" %scalarType
    samplemID = pylarc.read_row_major_matrix_from_file_matrixID(filename_rmm)
    pylarc.print_matrix_nonzeros_to_file_by_matrixID(samplemID,filename_nonzeros)
    pylarc.matrix_write_json_file_matrixID(samplemID,filename_json)
    if verbose:
        print "Printing the matrix we are testing for printing out nonzero values\n"
        pylarc.print_matrix_naive_by_matrixID(samplemID)
        

In [3]:
    verbose = 0
    
    # make CNOT
    if verbose:
        print "\nHere is the CNOT matrix\n"
    CNOT_arr = pylarc.buildArray([1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0])
    CNOTmID = pylarc.row_major_list_to_store_matrixID(CNOT_arr,level,level,dim_whole)
    pylarc.print_matrix_naive_by_matrixID(CNOTmID)

    # Calculate number of matrices created, then print part of matrix store
    num_matrices_made = pylarc.num_matrices_created()
    print "\n%d matrices have been created" %num_matrices_made
    start = end + 1
    end = num_matrices_made - 1
    filename = "../dat/out/cnot.%s.store" %scalarType
    pylarc.matrix_store_info_to_file(start,end,filename,"Loaded CNOT")

    # build Zero matrices
    if verbose:
        print "\nHere is the level 2 zero matrix\n"
    Z2_arr = pylarc.buildArray([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
    Z2mID = pylarc.row_major_list_to_store_matrixID(Z2_arr,level,level,dim_whole)
    pylarc.print_matrix_naive_by_matrixID(Z2mID)

    # build Identity matrices
    if verbose:
        print "\nHere is the level 2 identity matrix\n" 
    I2_arr = pylarc.buildArray([1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1])
    I2mID = pylarc.row_major_list_to_store_matrixID(I2_arr,level,level,dim_whole)
    pylarc.print_matrix_naive_by_matrixID(I2mID)

    # build a TOFFOLI
    if verbose:
        print "\nHere is the 3 bit TOFFOLI matrix with target 3rd.\n"
    # matrix_get_matrixID_from_panel is under construction
    TOFFOLImID= pylarc.matrix_get_matrixID_from_panel(I2mID,Z2mID,Z2mID,CNOTmID,3,3)
    pylarc.print_matrix_naive_by_matrixID(TOFFOLImID)
    filename = "../dat/out/toffoli.%s.naive" %scalarType
    pylarc.print_matrix_to_file_naive_by_matrixID(TOFFOLImID,filename)


    #  PLAYING WITH PREWRITTEN NONSQUARE MATRIX
    filename = "../dat/in/sample.1.2.%s.json" %scalarType
    if verbose:
        print "About to test read %s\n" %filename
    sampmID = pylarc.matrix_read_json_file_matrixID(filename)
    if verbose:
        print "We read in the json file\n"
    pylarc.print_matrix_naive_by_matrixID(sampmID)
    
    if verbose:
        print "does scalarM1_val print?"
    scalarM1_val = -1
    scalarM1mID = pylarc.matrix_get_matrixID_from_scalar(scalarM1_val)
    pylarc.print_matrix_naive_by_matrixID(scalarM1mID)
    
    if verbose:
        print "testing scalar_mult:"
    samp2mID = pylarc.scalar_mult_matrixID(scalarM1mID,sampmID)
    pylarc.print_matrix_naive_by_matrixID(samp2mID)
    
    if verbose:
        print "testing addition:"
    samp3mID = pylarc.matrix_add_matrixID(sampmID,samp2mID)
    pylarc.print_matrix_naive_by_matrixID(samp3mID)
    
    # save input matrixIDs for testing op store hash chains later
    in1_test_summID = sampmID
    in2_test_summID = samp2mID
    
    if verbose:
        print "testing adjoint:"
    samp3mID = pylarc.matrix_adjoint_matrixID(sampmID)
    pylarc.print_matrix_naive_by_matrixID(samp3mID)
    adjmID = samp3mID
    
    if verbose:
        print "testing non-square matrix mult:"
    samp4mID = pylarc.matrix_mult_matrixID(samp3mID,sampmID)
    pylarc.print_matrix_naive_by_matrixID(samp4mID)
    if verbose:
        print ""
    samp4mID = pylarc.matrix_mult_matrixID(sampmID,samp3mID)
    pylarc.print_matrix_naive_by_matrixID(samp4mID)
    if verbose:
        print "testing kron product:"
    samp4mID = pylarc.kronecker_product_matrixID(sampmID,sampmID)
    pylarc.print_matrix_naive_by_matrixID(samp4mID)
    

    if verbose:
        print "testing join:"
    samp4mID = pylarc.join_matrixID(sampmID,sampmID)
    pylarc.print_matrix_naive_by_matrixID(samp4mID)
    if verbose:
        print "testing stack:"
    samp4mID = pylarc.stack_matrixID(sampmID,sampmID)
    pylarc.print_matrix_naive_by_matrixID(samp4mID)



192 matrices have been created


In [4]:
    ##  TESTING DELETION
    print "\nPreparing to delete a matrix from the store.\n"
    filename = "../dat/out/temp.%s.json" %scalarType
    pylarc.matrix_write_json_file_matrixID(TOFFOLImID, filename)  
    pylarc.matrix_read_json_file_matrixID(filename)

    # Calculate number of matrices created, then print part of matrix store
    num_matrices_made = pylarc.num_matrices_created()
    print "\n%d matrices have been created" %num_matrices_made
    print "Previous range printed ended with matrixID %d\n" %end
    if (end == num_matrices_made-1) :
        print "Nothing new since last matrix store print\n"
    else :
        start = end + 1
        end = num_matrices_made - 1
        filename = "../dat/out/toffoli.%s.store" %scalarType
        pylarc.matrix_store_info_to_file(start,end,filename,"Loaded Toffoli")

    # get the hashID and print the hash chain corresponding to a matrix we are about to delete
    hashID = pylarc.matrix_hashID_from_matrixID(toffmID)
    comment = "hash chain before removal"
    filename = "../dat/out/hashChain.beforeMatrixRemove"
    pylarc.matrix_hash_chain_info_to_file(hashID, filename, comment)
    
    # Test deletion of a matrix
    print "Testing removal of matrix from the matrix store\n"
    num_matrices_made =  pylarc.num_matrices_created()
    end = num_matrices_made - 1
    filename = "../dat/out/toffoliYES.%s.store" %scalarType
    pylarc.matrix_store_info_to_file(0,end,filename,"Before Removed Toffoli")

    pylarc.remove_matrix_from_mat_store_by_matrixID(toffmID)
	
    filename = "../dat/out/toffoliNO.%s.store" %scalarType
    filename_json = "../dat/out/temp.%s.json" %scalarType
    print "\nDeleting the toffoli matrix with matrixID", toffmID,"from store, which had been read from %s\n"  %filename_json
    pylarc.matrix_store_info_to_file(0,end,filename,"Removed Toffoli")

    comment = "hash chain after removal"
    filename = "../dat/out/hashChain.afterMatrixRemove"
    pylarc.matrix_hash_chain_info_to_file(hashID, filename, comment)

    pylarc.list_op_names()
    


Preparing to delete a matrix from the store.


250 matrices have been created
Previous range printed ended with matrixID 191

Testing removal of matrix from the matrix store


Deleting the toffoli matrix with matrixID 183 from store, which had been read from ../dat/out/temp.Real.json



In [9]:
    
    # Test op store hash chains after deletion
	
    # print ops store report before deletion of a matrix
    # pylarc.op_store_report()
    
    # need op name to find the hash value so we can print op store hash chain
    op_name = "SUM"
    
    # get hash for an operation record and print the hash chain
    sum_hashID = pylarc.op_hashID_by_matrixIDs(in1_test_summID,in2_test_summID,op_name)
    if (sum_hashID != -1):
        pylarc.op_hash_chain_info_to_screen(sum_hashID,"op hash chain before deletion")
    else:
        print "invalid matrixID requested for op hash chain"
    
    # delete the first input matrix
    pylarc.remove_matrix_from_mat_store_by_matrixID(in1_test_summID)
    print "deleting matrix with matrixID", in1_test_summID
    
    # set a hold on a matrix by matrixID to see if it is immune to cleaning
    print "The matrixID of matrix to be held is", adjmID
    pylarc.set_hold_matrix_from_matrixID(adjmID)
    
    # clean the matrix store and print it again
    pylarc.clean_matrix_store()
    filename = "../dat/out/toffoliNOcleaned.%s.store" %scalarType
    pylarc.matrix_store_info_to_file(0,end,filename,"Removed Toffolli and cleaned matrix store")

    # clean the op store 
    # try one of three cleaning functions
    
    # clean a single op
    # pylarc.clean_single_op_from_string("SUM")
    
    # clean entire store
    # pylarc.clean_entire_op_store() 
    
    # clean entire store, one hash at a time
    for hash in range(1<<op_store_exp):
		pylarc.clean_op_hash_chain(hash) 
    
    # print same op hash chain again after deleting a matrix, holding a matrix and cleaning
    if (sum_hashID != -1):
        pylarc.op_hash_chain_info_to_screen(sum_hashID,"op hash chain after deletion and cleaning")
    else:
        print "invalid matrixID requested for op hash chain"
    

invalid matrixID requested for op hash chain
deleting matrix with matrixID 164
The matrixID of matrix to be held is 202
invalid matrixID requested for op hash chain


In [10]:
max_level
diag_hist = []
for i in range(max_level):
    diag_hist.append(pylarc.get_diag_hist(i))
diag_hist


[10, 6, 4, 4, 4, 4, 4, 4, 4, 4]

In [11]:
Earlier = [14, 6, 4, 4, 4, 4, 3, 2, 2, 1]

Earlier


[14, 6, 4, 4, 4, 4, 3, 2, 2, 1]

In [12]:
# plt.plot(Earlier)
# plt.plot(diag_hist)
# plt.show()
Earlier



[14, 6, 4, 4, 4, 4, 3, 2, 2, 1]

In [13]:
mat_store_exp
size_hash_table = 2**mat_store_exp
matrix_hash_chain_count = []
# pylarc.matrix_hash_chain_length(3)
print "mat_store_exp is %d" %mat_store_exp
print "size_hash_table is %d" %size_hash_table
hash_hist = [0, 0, 0, 0, 0]

for i in range(size_hash_table-1):
    count = pylarc.matrix_hash_chain_length(i)
    hash_hist[count] += 1
    # if count > 0:
    #    print "The number of entries in the %d hash chain of the table is %d"  %(i,count)
    matrix_hash_chain_count.append(count)
    
size_hash_table
if (size_hash_table <= 64):
    # show the array of hash chain lengths
    matrix_hash_chain_count
# plt.plot(matrix_hash_chain_count)
# plt.show()



mat_store_exp is 15
size_hash_table is 32768


In [14]:
hash_hist


[32602, 165, 0, 0, 0]